# LLM Data Cleansing

#### Loading Environment Variables

In [3]:
from dotenv import load_dotenv
import os

# loading .env file (no argument needed with default naming)
load_dotenv()

# setting api_key to the openai api key in file
LLM_API_KEY = os.getenv("OPENAI_API_KEY")